# Getting dask running

Using `dask` in a distributed way requires three components:

1. _worker_ processes to do the computation
2. a _scheduler_ process to coordinate the computation, by allocating (sub)tasks to workers, and moving data and results around
3. a _client_ to submit tasks to be carried out

To give your computations some _ooomph_, run the workers and scheduler on a powerful instance on EC2 or OpenStack. You can run the client on that instance too, or on your laptop if that's more convenient. These instructions assume you're running it on your laptop. You can also add more EC2 or OpenStack instances for additional workers, if you need even more _ooomph_.

### Set up the instance for the workers and scheduler

I have created an image on OpenStack called `ubuntu-18.04-python-3.7.0-dask` and an AMI on EC2 with the same name. The username for these is `ubuntu` rather than the `ec2-user` that you might be used to.

### Starting the scheduler

Log into your instance with `ssh`, and do port forwarding for ports 8786 and 8787 at the same time:

    ssh -L 8786:localhost:8786 -L 8787:localhost:8787 ubuntu@10.2.16.165

Port 8786 is for the workers and client will connect to the scheduler. Port 8786 is for the scheduler's web dashboard.

On your instance run

    cd dask_stuff
    pipenv run dask-scheduler

The scheduler will start and give you a message like this:

    distributed.scheduler - INFO - -----------------------------------------------
    distributed.scheduler - INFO - Clear task state
    distributed.scheduler - INFO -   Scheduler at:    tcp://172.16.0.27:8786
    distributed.scheduler - INFO -       bokeh at:                     :8787
    distributed.scheduler - INFO - Local Directory:    /tmp/scheduler-41ssh763
    distributed.scheduler - INFO - -----------------------------------------------



### Opening the web dashboard

You can now open the web dashboard by going here [http://localhost:8787](http://localhost:8787).

### Starting the workers

With another `ssh` session, or using `screen` or `tmux` magic, run something like following:

    cd dask_stuff/dask_workers
    pipenv run dask-worker --nprocs 12 --nthreads 1 172.16.0.27:8786 --memory-limit 4GB

Note the following parts of the command:

- the `nprocs` part determines how many worker processes will be started
- the `nthreads` part determines how many threads will run within each worker process (I would suggest just setting this to 1, because of python's Global Interpreter Lock bollocks)
- the IP and port number `172.16.0.27:8786` need to match what the scheduler output when it was started (and if you want to run additional workers on additional instances, this is how they will know where the scheduler is)
- the `memory-limit` part sets how much memory each worker process is allocated

On the scheduler's stdout you will see a message like this for each worker that joins:

    distributed.scheduler - INFO - Register tcp://172.16.0.27:37375
    distributed.scheduler - INFO - Starting worker compute stream, tcp://172.16.0.27:37375
    distributed.core - INFO - Starting established connection

You can also check in the "Workers" tab of the web dashboard, to see your lovely workers read to work for you.

### Starting the client

You're now ready to start a client and do some work. You can run the client on your own laptop, where you can install `dask` like this:

    pipenv install "dask[complete]"

Try to have the same versions of python and `dask` on the different machines. One thing I have found that definitely caused problems was trying to mix machines running python 3.6.something with machines running python 3.7.something. (This is why the AMI I've made contains a python 3.7.0 built from source.)


The other example notebooks included will show you how to run a client :) Happy `dask`ing!